In [1]:
import os

def find_and_remove_nonstandard_labels(base_dir):
    nonstandard_entries = {}
    total_nonstandard_files = 0
    removed_labels = 0
    removed_images = 0

    for split in ['train', 'valid', 'test']:
        label_dir = os.path.join(base_dir, split, 'labels')
        image_dir = os.path.join(base_dir, split, 'images')

        if not os.path.exists(label_dir):
            continue

        for filename in os.listdir(label_dir):
            if filename.endswith('.txt'):
                filepath = os.path.join(label_dir, filename)
                with open(filepath, 'r') as file:
                    lines = file.readlines()

                bad_lines = []
                for i, line in enumerate(lines):
                    parts = line.strip().split()
                    if len(parts) != 5:
                        bad_lines.append((i + 1, line.strip()))

                if bad_lines:
                    total_nonstandard_files += 1

                    # Guess image name
                    base_name = os.path.splitext(filename)[0]
                    image_name = None
                    image_path = None
                    for ext in ['.jpg', '.png', '.jpeg']:
                        potential_image_path = os.path.join(image_dir, base_name + ext)
                        if os.path.exists(potential_image_path):
                            image_name = base_name + ext
                            image_path = potential_image_path
                            break

                    # Save entry
                    nonstandard_entries[f"{split}/labels/{filename}"] = {
                        "image": image_name or "Image not found",
                        "issues": bad_lines
                    }

                    # 🗑️ Remove label file
                    os.remove(filepath)
                    removed_labels += 1

                    # 🗑️ Remove image file if found
                    if image_path:
                        os.remove(image_path)
                        removed_images += 1

    return nonstandard_entries, total_nonstandard_files, removed_labels, removed_images

# Dataset root
dataset_root = r"C:\\Users\\ADMIN\\Desktop\\Project_ANPR\\Plate_Detection_Dataset"

# Run and clean
nonstandard_labels, total, deleted_labels, deleted_images = find_and_remove_nonstandard_labels(dataset_root)

# Output
if nonstandard_labels:
    print(f"⚠️ Found {total} label files with non-standard YOLO lines.")
    print(f"🗑️ Removed {deleted_labels} label files and {deleted_images} image files.\n")

    for label_file, info in nonstandard_labels.items():
        print(f"Label File: {label_file}")
        print(f"Image File: {info['image']}")
        for line_no, content in info['issues']:
            print(f"  Line {line_no}: {content}")
        print()
else:
    print("✅ All label files are valid with exactly 5 values per line.")


✅ All label files are valid with exactly 5 values per line.
